### <center> Court Judgment General and Technical Summarizations and Legistlation Used List </center>
    
This code uses **LangChain** to:
* Use an **`UnstructuredXMLLoader`** to read a Court Judgment from an XML Document.
    * These documents tent to be very large.
* Summarize the document with a MapReduce Chain which in turn: 
    * Uses a **`ReduceDocumentsChain`** to split the document into smaller parts
    * Uses an **`LLMchain`** to summarize each part
        * There are two versions of this: 
            * Make a summary with simple language aimed at general audience
            * Make a summary with technical language aimed at legal experts
    * Uses a **`StuffDocumentsChain`** to group the summaries together
    * Uses the **`LLMChain`** again to summarize the grouped summaries
* Then we use another **`LLMChain`** to read the technical summary to list and explain the legislation used in this Court Judgement.

### 0. Imports and API Keys Setup

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredXMLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain.prompts import ChatPromptTemplate

### 1. Load and Split XML Court Judgement Document

In [3]:
xml_loader = UnstructuredXMLLoader(file_path="uksc_2023_37.xml")

In [4]:
court_judgment = xml_loader.load()

In [5]:
print(f"Number of Documents:{len(court_judgment)}")

Number of Documents:1


In [6]:
print(f"Word cound of document content: {len(court_judgment[0].page_content.split()):,}")

Word cound of document content: 4,224


In [7]:
text_splitter = CharacterTextSplitter(chunk_size=1500)

In [8]:
split_court_judgment = text_splitter.create_documents([court_judgment[0].page_content])

Created a chunk of size 3854, which is longer than the specified 1500


In [9]:
len(split_court_judgment)

20

### 2. Summarize Legal Document with a Map-Reduce Chain

#### Map Chain
* Two versions: 
    1. With a general audience prompt
    2. With a legal expert prompt

In [10]:
llm = ChatOpenAI(temperature=0, model='gpt-4-1106-preview')

In [11]:
map_template_general = ''' 
Write a concise summary of the following legal document in simple terms so that it is accessible to a general audience: 
{legal_document} 
Summary:
'''
map_prompt_general = PromptTemplate.from_template(map_template_general)
map_chain_general = LLMChain(prompt=map_prompt_general, llm=llm)

In [12]:
map_template_legal = ''' 
Write a concise summary of the following legal document so that it is accessible to a legal expert: 
{legal_document} 
Summary:
'''
map_prompt_legal = PromptTemplate.from_template(map_template_legal)
map_chain_legal = LLMChain(prompt=map_prompt_legal, llm=llm)

### Reduce Chain

In [13]:
reduce_template = ''' 
Summarize the following set of summaries with all key details:
{doc_summaries} 
Summary:
'''
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, 
                        prompt=reduce_prompt)

stuff_chain = StuffDocumentsChain(llm_chain=reduce_chain, 
                                  document_variable_name="doc_summaries")

reduce_chain = ReduceDocumentsChain(combine_documents_chain=stuff_chain)

### Mab Reduce Chain
* Two versions:
    1. Using the map_chain_general
    2. Using the map_chain_legal

In [14]:
map_reduce_chain_general = MapReduceDocumentsChain(
    llm_chain=map_chain_general,
    document_variable_name="legal_document",
    reduce_documents_chain=reduce_chain
)

In [15]:
map_reduce_chain_legal = MapReduceDocumentsChain(
    llm_chain=map_chain_legal,
    document_variable_name="legal_document",
    reduce_documents_chain=reduce_chain
)

In [16]:
summary_general = map_reduce_chain_general.run(split_court_judgment)

In [17]:
summary_legal = map_reduce_chain_legal.run(split_court_judgment)

In [18]:
print(summary_general)

The UK Supreme Court case involves HMRC and Vermilion Holdings Ltd, focusing on the tax treatment of employment-related securities options under the Income Tax (Earnings and Pensions) Act 2003. Vermilion offered share options as compensation during a 2006 financial restructuring, which were later altered during a 2007 rescue funding exercise. Mr. Marcus Noble, associated with Quest, exercised an option in 2016, leading HMRC to charge Vermilion income tax and National Insurance Contributions, treating the profits as employment income.

The legal issue revolved around whether the share options were granted "by reason of an employment," affecting their tax status. The First-tier Tribunal and higher courts debated the application of a "deeming provision" that assumes options provided by an employer are related to employment. The Inner House had a majority ruling that the 2007 option was not employment-related, while the Lord President dissented.

The Supreme Court, with a judgment written 

In [19]:
print(f"General Summary Word Cound: {len(summary_general.split()):,}")

General Summary Word Cound: 280


In [20]:
print(summary_legal)

The legal case involves Vermilion Holdings Ltd and HMRC, focusing on the interpretation of section 471 of the Income Tax (Earnings and Pensions) Act 2003, which deals with the taxation of employment-related securities options. Vermilion Holdings was created after the restructuring of Vermilion Software Ltd in 2006. In 2007, during a fundraising event, supplier options were granted to Quest Advantage Ltd and Dickson Minto, which HMRC later challenged as taxable under employment income.

The case hinged on whether the securities option granted to Mr. Noble was employment-related as per section 471 of ITEPA. The First-tier Tribunal (FTT) and the Inner House had differing views on whether the option was granted by reason of employment and the application of the deeming provision in section 471(3). The FTT found that the option was not solely related to employment, while the Upper Tribunal allowed HMRC's case. The Inner House, by majority, sided with the FTT, but the Lord President dissente

In [22]:
print(f"General Summary Word Cound: {len(summary_legal.split()):,}")

General Summary Word Cound: 333


### 3. What Law was Used

In [39]:
template = ''' You are a legal expert in UK law. 
               Provide a list with simple explanations wich UK legistlation was interpreted in this court judgment summary:\n{document} '''
prompt = ChatPromptTemplate.from_template(template)
chain = LLMChain(llm=llm,
                 prompt=prompt)

In [40]:
result = chain.run(summary_legal)

In [41]:
print(result)

Based on the court judgment summary provided, the following UK legislation and legal principles were interpreted:

1. **Income Tax (Earnings and Pensions) Act 2003 (ITEPA 2003)** - Specifically, section 471 of ITEPA 2003 was the central piece of legislation under interpretation. This section deals with the taxation of employment-related securities options. The case focused on whether the securities option granted was employment-related and the application of the deeming provision within this section.

2. **Deeming Provisions** - The interpretation of deeming provisions within the legislation was a key issue. Deeming provisions are legal constructs that treat something as if it were something else for specific purposes, even if it is not actually the case. The courts had to consider how to apply these provisions without leading to absurd or unreasonable outcomes.

3. **Case Law: Price v Revenue and Customs Comrs** - This case was referenced in the judgment summary and influenced the Fir

Reference: 
* [Example Court Judgement Used](https://caselaw.nationalarchives.gov.uk/uksc/2023/37#download-options)
* [LangChain Summarization Documentation](https://python.langchain.com/docs/use_cases/summarization)
* [How to summarize Large Documents with LangChain and OpenAI in Python](https://harikirankante.hashnode.dev/how-to-summarize-large-documents-using-langchain-and-openai-in-python)